In [1]:
import matplotlib.pyplot as plt
import numpy as np
import pandas as pd
from datetime import datetime
import ast
%config InlineBackend.figure_format = 'retina'
plt.rcParams['font.sans-serif'] = 'SimHei'
# 用于正常显示符号
plt.rcParams['axes.unicode_minus'] = False

In [2]:
df=pd.read_csv('./data/task1_dataset.csv')

In [3]:
df.head()

,uid,d,t,x,y
0,0,0,1,79,86
1,0,0,2,79,86
2,0,0,8,77,86
3,0,0,9,77,86
4,0,0,19,81,89


In [4]:
print(len(df))

111535175


In [5]:
df=df.drop(df[df['x']==999].index)

In [6]:
df['p']=(df['x']-1)*200+df['y']-1

In [7]:
df.head()

,uid,d,t,x,y,p
0,0,0,1,79,86,15685
1,0,0,2,79,86,15685
2,0,0,8,77,86,15285
3,0,0,9,77,86,15285
4,0,0,19,81,89,16088


In [8]:
print(max(df['uid']))

99999


## 因为内存不够，所以得分两批数据跑

In [9]:
df=df[df['uid']>=50000]

In [10]:
df_train=df[df['d']<60]
df_valid=df[df['d']>=60]

In [11]:
df_train=df_train[['uid','p']]
df_valid=df_valid[['uid','p']]

In [12]:
data=df_train.values.tolist()

In [13]:
map_user1=[]
map_user2=[]
user_num=np.zeros((max(df['uid']))+1,dtype=np.int32)
for i in range((max(df['uid']))+1):
    map_user1.append(dict())
    map_user2.append(dict())

In [14]:
for i in range(len(data)):
    if data[i][1] not in map_user1[data[i][0]].keys():
        map_user1[data[i][0]][data[i][1]]=user_num[data[i][0]]
        map_user2[data[i][0]][user_num[data[i][0]]]=data[i][1]
        user_num[data[i][0]]+=1

In [15]:
cnt=[]
for i in range(max(df_train['uid'])+1):
    cnt.append(np.zeros((user_num[i],user_num[i])))

In [16]:
for i in range(len(data)):
    if i>0 and data[i][0]==data[i-1][0]:
        cnt[data[i][0]][map_user1[data[i][0]][data[i-1][1]]][map_user1[data[i][0]][data[i][1]]]+=1

In [17]:
max_num_cnt=[]
for i in range(max(df_train['uid'])+1):
    max_num_cnt.append([])
for i in range(max(df_train['uid'])+1):
    for j in range(user_num[i]):
        max_num_cnt[i].append(np.argmax(cnt[i][j]))

In [32]:
last_p=[]
for i in range(len(data)):
    if i>0 and data[i][0]!=data[i-1][0]:
        last_p.append(map_user1[data[i-1][0]][data[i-1][1]])
    if i==len(data)-1:
        last_p.append(map_user1[data[i][0]][data[i][1]])

In [33]:
print(max_num_cnt[0])

[]


In [34]:
predict_num=np.zeros(max(df_valid['uid'])+1,dtype=np.int32)
data_valid=df_valid.values.tolist()

In [35]:
for i in range(len(data_valid)):
    predict_num[data_valid[i][0]]+=1

In [36]:
predict_p=[]
min_uid=min(df_valid['uid'])
max_uid=max(df_valid['uid'])
for i in range(min_uid,max_uid+1):
    for j in range(predict_num[i]):
        pre_point=max_num_cnt[i][last_p[i-min_uid]]
        last_p[i-min_uid]=pre_point
        predict_p.append(map_user2[i][pre_point])

In [37]:
df_valid=df[df['d']>=60]

In [38]:
df_valid['pred_p']=predict_p

/root/miniconda3/lib/python3.6/site-packages/ipykernel_launcher.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  """Entry point for launching an IPython kernel.


In [39]:
df_valid['pred_x']=df_valid['pred_p']//200+1
df_valid['pred_y']=df_valid['pred_p']%200+1

/root/miniconda3/lib/python3.6/site-packages/ipykernel_launcher.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  """Entry point for launching an IPython kernel.
/root/miniconda3/lib/python3.6/site-packages/ipykernel_launcher.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  


In [40]:
df_valid.head()

,uid,d,t,x,y,p,pred_p,pred_x,pred_y
56555275,50000,60,12,163,62,32461,32853,165,54
56555276,50000,60,13,160,63,31862,32852,165,53
56555277,50000,60,14,152,83,30282,32853,165,54
56555278,50000,60,15,152,83,30282,32852,165,53
56555279,50000,60,16,152,83,30282,32853,165,54


In [41]:
def calc_geobleu_t(df):
    reference = df[["d","t","x","y"]].to_numpy()
    generated = df[["d","t","pred_x","pred_y"]].to_numpy()
    return geobleu.calc_geobleu_single(generated.tolist(), reference.tolist()) 。

In [42]:
import geobleu
from joblib import Parallel, delayed
from tqdm import tqdm
result=[]
grouped = df_valid[["uid", "d", "t", "x", "y", "pred_x", "pred_y"]].groupby("uid")
results = Parallel(n_jobs=-1)(delayed(calc_geobleu_t)(df) for _, df in tqdm(grouped))

100%|██████████| 30000/30000 [10:23<00:00, 48.13it/s]


In [43]:
print(sum(results)/len(results))

0.18854974102426672


In [44]:
def calc_dtw_t(df):
    reference = df[["d","t","x","y"]].to_numpy()
    generated = df[["d","t","pred_x","pred_y"]].to_numpy()
    return geobleu.calc_dtw_single(generated.tolist(), reference.tolist()) 

In [45]:
import geobleu
from joblib import Parallel, delayed
from tqdm import tqdm
result=[]
grouped = df_valid[["uid", "d", "t", "x", "y", "pred_x", "pred_y"]].groupby("uid")
results = Parallel(n_jobs=-1)(delayed(calc_dtw_t)(df) for _, df in tqdm(grouped))

100%|██████████| 30000/30000 [02:32<00:00, 197.24it/s]


In [46]:
print(sum(results)/len(results))

26.72210537216688


In [47]:
print(len(results))

30000


In [ ]:
## 第一批 geoblue： 0.1754487034314798  dtw：27.732926915321535  50000      
## 第二批 geoblue： 0.18854974102426672  dtw：26.72210537216688  30000

In [48]:
(0.1754487034314798*50000+0.18854974102426672*30000)/80000

0.1803615925287749

In [49]:
(27.732926915321535*50000+26.72210537216688*30000)/80000

27.35386883663854